In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
from pathlib import Path

### Pandas makes loading data easy

#### Let's pull down a bit of weather data from Central Park

In [ ]:
url = 'https://raw.githubusercontent.com/toddwschneider/nyc-taxi-data/master/data/central_park_weather.csv'

In [ ]:
df = pd.read_csv(url, parse_dates=['DATE'])

In [ ]:
df.head()

### Grouped aggregations (`GROUP BY` in SQL) are very easy to write

#### Average max temperature per year

In [ ]:
avg = df.groupby(df.DATE.dt.year).TMAX.mean()
avg.plot()

### Average monthly high, low temperature

In [ ]:
df

In [ ]:
resamp = df.set_index('DATE').resample('D')
ax = resamp.TMAX.mean().plot()
resamp.TMIN.mean().plot(ax=ax)

### What's the relationship between # of rides and weather?

```sh
wget https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2017-01.csv
```

In [ ]:
!du -h /home/phillip/fhv_tripdata_2017-01.csv

In [ ]:
%%time
taxi = pd.read_csv(Path('~') / 'fhv_tripdata_2017-01.csv', parse_dates=['Pickup_date'], usecols=['locationID', 'Pickup_date'])

In [ ]:
pickup_date = taxi.Pickup_date
jan_taxi = taxi[pickup_date.dt.year.eq(2017) & pickup_date.dt.month.eq(1)]

In [ ]:
rides_per_day = (
    jan_taxi.set_index('Pickup_date')
            .locationID
            .resample('D')
            .count())
rides_per_day

In [ ]:
temp = resamp.TMAX.mean().loc['2017-01':'2017-01-31']
frame = temp.to_frame().assign(rides=rides_per_day)
frame.plot.scatter('TMAX', 'rides');